### Задача

Используя доступный набор данных о посещении страниц у одной части пользователей, сделать прогноз относительно **пола и возрастной категории** другой части пользователей. Угадывание (hit) - правильное предсказание и пола, и возрастной категории одновременно.

Мы не ограничиваем вас в выборе инструментов и методов работы с данными. Используйте любые эвристики, внешние источники, парсинг контента страниц — всё, что поможет вам выполнить задачу. Единственное ограничение — никаких ручных действий. Руками проставлять классы нельзя.

Поскольку это ваш проект, который мы наверняка захотите показать другим, уделите его оформлению достаточно времени. Мы рекомендуем сделать весь проект в этом ноутбуке. Снизу, под заданием, вы сможете описать ваше решение.

⏰ **Дедлайн: 06 мая 2018, 23:59**

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import re
import os, sys
import json
from urllib.parse import urlparse
from urllib.request import urlretrieve, unquote
import datetime

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
from xgboost import XGBClassifier
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
from collections import Counter
import seaborn as sns
from sklearn.metrics import accuracy_score
plt.style.use('ggplot')

# Import

In [3]:
# First, we download the input data for the project:

data_dir = '/data/home/artem.merinov/project01/'
filename = 'gender_age_dataset.txt'
file_path = '/'.join([data_dir,filename])
url = 'http://data.newprolab.com/data-newprolab-com/project01/' + filename

if not os.path.isdir(data_dir): os.mkdir(data_dir)
if not os.path.isfile(file_path):
    print('Downloading ' + filename + '...')
    urlretrieve(url, file_path)
    print('Download completed')

# Wait until you see that all files have been downloaded.
print('OK')

OK


In [4]:
df = pd.read_csv(file_path, sep='\t')

In [5]:
df.head()

,gender,age,uid,user_json
0,F,18-24,d50192e5-c44e-4ae8-ae7a-7cfe67c8b777,"{""visits"": [{""url"": ""http://zebra-zoya.ru/2000..."
1,M,25-34,d502331d-621e-4721-ada2-5d30b2c3801f,"{""visits"": [{""url"": ""http://sweetrading.ru/?p=..."
2,F,25-34,d50237ea-747e-48a2-ba46-d08e71dddfdb,"{""visits"": [{""url"": ""http://ru.oriflame.com/pr..."
3,F,25-34,d502f29f-d57a-46bf-8703-1cb5f8dcdf03,"{""visits"": [{""url"": ""http://translate-tattoo.r..."
4,M,>=55,d503c3b2-a0c2-4f47-bb27-065058c73008,"{""visits"": [{""url"": ""https://mail.rambler.ru/#..."


В файле есть 4 поля:
* **gender** - пол, принимающий значения `M` (male - мужчина), `F` (female - женщина), `-` (пол неизвестен);
* **age** - возраст, представленный в виде диапазона x-y (строковый тип), или `-` (возрастная категория неизвестна);
* **uid** - идентификатор пользователя, строковая переменная;
* **user_json** - поле json, в котором содержатся записи о посещении сайтов этим пользователем `(url, timestamp)`.

# Feature engineering

**Распарсим json-ы для всего датафрейма**

In [6]:
df['user_json_parsed'] = df['user_json'].apply(json.loads)

In [7]:
df.head(2)

,gender,age,uid,user_json,user_json_parsed
0,F,18-24,d50192e5-c44e-4ae8-ae7a-7cfe67c8b777,"{""visits"": [{""url"": ""http://zebra-zoya.ru/2000...",{'visits': [{'url': 'http://zebra-zoya.ru/2000...
1,M,25-34,d502331d-621e-4721-ada2-5d30b2c3801f,"{""visits"": [{""url"": ""http://sweetrading.ru/?p=...",{'visits': [{'url': 'http://sweetrading.ru/?p=...


**И для каждого пользователя вытащим**
    
    список урлов 
    список доменов
    список времени посещения сайтов    

In [8]:
%%time
def url2domain(url):
    url = re.sub('(http(s)*://)+', 'http://', url)
    parsed_url = urlparse(unquote(url.strip()))
    if parsed_url.scheme not in ['http','https']: 
        return None
    netloc = re.search("(?:www\.)?(.*)", parsed_url.netloc).group(1)
    if netloc is not None: 
        return netloc
    return None

def get_visits_info(df):
    urls_list, domains_list, time_list = [], [], []
    for uid in range(len(df)):
        visits = df.iloc[uid]['user_json_parsed']['visits']
        urls_per_user, time_per_user, domains_per_user = [], [], []
        for visit in visits:        
            urls_per_user.append(visit['url'])
            domains_per_user.append(url2domain(visit['url']))
            time_per_user.append(datetime.datetime.fromtimestamp(visit['timestamp'] / 1e3))            
        urls_list.append(urls_per_user)
        domains_list.append(domains_per_user)
        time_list.append(time_per_user)
    df = df.copy()
    df['urls'] = urls_list
    df['domains'] = domains_list
    df['timestamp'] = time_list
    return df

df = get_visits_info(df)

CPU times: user 1min 40s, sys: 2.14 s, total: 1min 42s
Wall time: 1min 42s


In [9]:
df.head(2)

,gender,age,uid,user_json,user_json_parsed,urls,domains,timestamp
0,F,18-24,d50192e5-c44e-4ae8-ae7a-7cfe67c8b777,"{""visits"": [{""url"": ""http://zebra-zoya.ru/2000...",{'visits': [{'url': 'http://zebra-zoya.ru/2000...,[http://zebra-zoya.ru/200028-chehol-organayzer...,"[zebra-zoya.ru, news.yandex.ru, sotovik.ru, ne...","[2014-12-27 16:49:04.068000, 2015-03-18 11:11:..."
1,M,25-34,d502331d-621e-4721-ada2-5d30b2c3801f,"{""visits"": [{""url"": ""http://sweetrading.ru/?p=...",{'visits': [{'url': 'http://sweetrading.ru/?p=...,"[http://sweetrading.ru/?p=900, http://sweetrad...","[sweetrading.ru, sweetrading.ru, sweetrading.r...","[2014-12-28 01:04:46.224000, 2014-12-28 01:04:..."


**Создадим новые признаки**
    
    night_visit_rate
    morning_visit_rate
    daytime_visit_rate
    evening_visit_rate
   

In [10]:
%%time
def time2daytime(df):
    night_visit_rate, morning_visit_rate, daytime_visit_rate, evening_visit_rate = [], [], [], []

    for uid in range(len(df)):
        ts_per_uid = df.iloc[uid]['timestamp']    
        night_visits, morning_visits, daytime_visits, evening_visits = 0, 0, 0, 0
        for ts in ts_per_uid:               

            if 0 <= ts.hour < 6:
                night_visits += 1
            elif 6 <= ts.hour < 12:
                morning_visits += 1
            elif 12 <= ts.hour < 18:
                daytime_visits += 1
            elif 18 <= ts.hour <= 23:
                evening_visits += 1

        night_visit_rate.append(night_visits / len(ts_per_uid))
        morning_visit_rate.append(morning_visits / len(ts_per_uid))
        daytime_visit_rate.append(daytime_visits / len(ts_per_uid))
        evening_visit_rate.append(evening_visits / len(ts_per_uid))
    
    df = df.copy()
    df['night_visit_rate'] = night_visit_rate
    df['morning_visit_rate'] = morning_visit_rate
    df['daytime_visit_rate'] = daytime_visit_rate
    df['evening_visit_rate'] = evening_visit_rate
    return df

df = time2daytime(df)

CPU times: user 7.27 s, sys: 36.1 ms, total: 7.31 s
Wall time: 7.31 s


In [11]:
df.head(2)

,gender,age,uid,user_json,user_json_parsed,urls,domains,timestamp,night_visit_rate,morning_visit_rate,daytime_visit_rate,evening_visit_rate
0,F,18-24,d50192e5-c44e-4ae8-ae7a-7cfe67c8b777,"{""visits"": [{""url"": ""http://zebra-zoya.ru/2000...",{'visits': [{'url': 'http://zebra-zoya.ru/2000...,[http://zebra-zoya.ru/200028-chehol-organayzer...,"[zebra-zoya.ru, news.yandex.ru, sotovik.ru, ne...","[2014-12-27 16:49:04.068000, 2015-03-18 11:11:...",0.000000,0.800000,0.200000,0.000000
1,M,25-34,d502331d-621e-4721-ada2-5d30b2c3801f,"{""visits"": [{""url"": ""http://sweetrading.ru/?p=...",{'visits': [{'url': 'http://sweetrading.ru/?p=...,"[http://sweetrading.ru/?p=900, http://sweetrad...","[sweetrading.ru, sweetrading.ru, sweetrading.r...","[2014-12-28 01:04:46.224000, 2014-12-28 01:04:...",0.147059,0.029412,0.205882,0.617647


# Site parsing

In [12]:
from bs4 import BeautifulSoup
import requests
from lxml.html import fromstring

**Пропарсим содержимое урлов и вытащим title**

In [18]:
def get_title_from_url(df):
    titles_list = []
    for count, uid in enumerate(range(len(df))):        
        # выберем индексы уникальных доменов
#         user_domains_unique, indices = np.unique(df.iloc[uid]['domains'], return_index=True)
        user_urls = df.iloc[uid]['urls']
#         user_urls_unique = [df.iloc[uid]['urls'][i] for i in indices]
        titles_per_user = []
        for time_idx, url in enumerate(user_urls):
#         for url in user_urls_unique:
            # try except нужен, т.к. могут быть connection problems
            try:
                r = requests.get(url, timeout=1)
                if r.encoding != 'ISO-8859-1':           
                    html = BeautifulSoup(r.text, 'lxml')
                    title = html.title.text
                    titles_per_user.append(title)
                    print(count, url)
                    print(title, df.iloc[uid]['timestamp'][time_idx])
            except:
                print('------Missed-------')
                
    titles_list.append(titles_per_user)
#     df_with_title = df.copy()
#     df_with_title['titles'] = titles_list
    print(df.iloc[uid][['gender', 'age']])
    return titles_list

i = random.randint(1, 30000)
get_title_from_url(df[i:i+1])

0 http://www.dns-shop.ru/catalog/i192275/smartfon-huawei-ascend-g750-honor-3x-55-8gb-white.html
404 Not Found 2014-12-28 20:37:31.015000
0 http://www.dns-shop.ru/catalog/i197632/smartfon-dns-s4509-45-4gb-white.html
404 Not Found 2014-12-28 19:46:14.944000
0 http://www.widewallpapers.ru/index.php?p=static&page=interior-wallpapers-3
404 Not Found 2014-12-27 10:13:27.911000
0 http://www.widewallpapers.ru/index.php?p=static&page=interior-wallpapers-2
404 Not Found 2014-12-27 10:13:13.548000
0 http://www.hdwallpapers.in/planes-desktop-wallpapers.html
Military Wallpapers of Army, Warships, Jet Fighters - Page 1 2014-12-27 10:07:31.545000
0 http://www.hdwallpapers.in/inspirational-desktop-wallpapers.html
Inspirational Wallpapers - Page 1 - HD Wallpapers 2014-12-27 10:07:18.565000
0 http://www.hdwallpapers.in/1366x768_hd-wallpapers-r.html
1366x768 HD Resolution Wallpapers - Page 1  2014-12-27 10:06:57.771000
0 http://chuvashia.petovod.ru/cheboksary/sell/cats/burmanskaya/burmanskie-kotyata-2652

[['404 Not Found',
  '404 Not Found',
  '404 Not Found',
  '404 Not Found',
  'Military Wallpapers of Army, Warships, Jet Fighters - Page 1',
  'Inspirational Wallpapers - Page 1 - HD Wallpapers',
  '1366x768 HD Resolution Wallpapers - Page 1 ',
  'Продаю.Бурманские котята!-Чебоксары- частные бесплатные объявления на сайте Petovod.ru ',
  'Новогодние игрушки своими руками / Pinme',
  'Ехали медведи на велосипеде..."Тараканище" К.Чуковский (стих и мультик) | Мама, папа и дитя',
  '\r\n\tKaspersky Anti-Virus 2015\r\n',
  'Бесплатная музыка, mp3 на МУЗ-ТВ, слушать бесплатно онлайн',
  'Поиск',
  'Бесплатная музыка, mp3 на МУЗ-ТВ, слушать бесплатно онлайн',
  'МУЗ-ТВ Чарт - МУЗ-ТВ',
  'Смотреть каталог Орифлейм 6 2015, страница 1 :: Онлайн каталог Oriflame',
  'Смотреть каталог Орифлэйм 8 2018 онлайн, новый каталог Oriflame 7 2018, каталог Орифлейм 6 2018']]

In [19]:
r = requests.get('http://povar.ru/recipes/makarony_s_ovoshami-18185.html')
html = BeautifulSoup(r.text, 'lxml')
html.title.text

'Макароны с овощами - пошаговый рецепт с фото на Повар.ру'

# Bag of sites

In [167]:
def site_counter(df, top_num):
    lst = []
    for i in range(len(df)):
        lst.append(df.iloc[i]['domains'])
    flat_list = [item for sublist in lst for item in sublist]
    
    c = Counter()
    for word in flat_list:
        c[word] += 1
    
    return c.most_common(top_num)

site_counter(df, top_num=5)

[('avito.ru', 466510),
 ('smotri.com', 207694),
 ('24open.ru', 91862),
 ('loveplanet.ru', 91054),
 ('mail.rambler.ru', 81633)]

In [168]:
%%time
def make_bag_of_sites_df(feat_num):
    top_n = [val[0] for val in site_counter(df, feat_num)]    
    vec_top_n = []
    for i in range(len(df)):
        uid_domains = df.iloc[i]['domains']
        uid_vec_top_n = []
        for d in top_n:
            uid_vec_top_n.append(uid_domains.count(d))
        vec_top_n.append(uid_vec_top_n)
    return pd.DataFrame(vec_top_n, columns=top_n)

bag_of_sites = make_bag_of_sites_df(feat_num=300)

In [169]:
df_res = pd.concat([df[['gender', 'age']], bag_of_sites], axis=1)

In [170]:
df_res.head(3)

,gender,age,avito.ru,smotri.com,24open.ru,loveplanet.ru,mail.rambler.ru,youtube.com,yandex.ru,vk.com,...,ru.jobrapido.com,date.bluesystem.ru,speedtest.net,retest.me,tempfile.ru,vtraxe.com,ridus.ru,live.russia.tv,i6.webware.ru,love.nnovgorod.net
0,F,18-24,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,M,25-34,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,F,25-34,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Деление на train и test сеты, обучение модели, предсказания для test-сета

In [14]:
train_df = df_res[~((df_res.gender == '-') & (df_res.age == '-'))]
test_df = df_res[(df_res.gender == '-') & (df_res.age == '-')]
print('train/test split:', len(train_df), len(test_df))

train/test split: 36138 5000


#### Посмотрим на сбалансированность выборки по целевым переменным

In [18]:
# fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, sharex=False, sharey=True, figsize=(16,4))

# ax1.set_title('Gender distribution')
# ax1.bar(x=['F', 'M'], 
#         height=(len(train_df[train_df['gender'] == 'F']), len(train_df[train_df['gender'] == 'M'])),
#         width=0.6, color='c', alpha=0.6);

# ax2.set_title('Age distribution')
# ax2.bar(x=train_df['age'].unique(), 
#         height=train_df.groupby('age').count().reset_index()['gender'].values,
#         width=0.6, color='g', alpha=0.6);

# plt.tight_layout()

In [19]:
train_df = train_df.drop('age', axis=1)
test_df = test_df.drop(['gender', 'age'], axis=1)

In [20]:
# f - 0, m -1
train_df['gender'] = pd.Series(train_df['gender']).astype('category').cat.codes

In [21]:
train_df.head()

,gender,avito.ru,smotri.com,24open.ru,loveplanet.ru,mail.rambler.ru,youtube.com,yandex.ru,vk.com,ebay.com,...,ru.jobrapido.com,date.bluesystem.ru,speedtest.net,retest.me,tempfile.ru,vtraxe.com,ridus.ru,live.russia.tv,i6.webware.ru,love.nnovgorod.net
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,46,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [22]:
X = train_df.drop(['gender'], axis=1)
y = train_df['gender']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [51]:
clf = RandomForestClassifier(n_jobs=20, n_estimators=1000, max_depth=None, min_samples_split=2, random_state=0)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

In [52]:
accuracy_score(y_test, y_pred)

0.6317100166021029

In [ ]:
sorted(list(zip(clf.feature_importances_, X_train.columns)), reverse=True)[:15]

In [53]:
# model = XGBClassifier(max_depth=10, learning_rate=0.05, n_estimators=500)
# model.fit(X_train, y_train)

In [54]:
# y_pred = model.predict_proba(X_test)

# from sklearn.metrics import accuracy_score
# accuracy_score(y_test, y_pred)

### Сохранение модели

Обучать модель вы можете в ноутбуке - это удобно. А после того как модель обучилась и стала выдавать приемлемое качество на вашей валидационной выборке, сохраните ее в отдельном файле (например, используя pickle):

In [ ]:
#Сохранить модель, которая содержится в переменной vectorizer
import pickle
with open('TfidfVectorizer.pickle', 'wb') as f:
    pickle.dump(vectorizer, f)


Однако, не забудьте сохранить и код генерации признаков и обучения модели - это нужно для воспроизводимости результатов.

### Обработка тестовых данных и формат вывода результатов

Помимо того, что у вас должна получиться точная модель, вам нужно уложиться в SLA (service-level agreement). Всё почти как по-настоящему. Результатом вашей работы в данном случае будет не выходной файл, в котором вы всё посчитали для скрытой выборки, а скрипт, который мы будем запускать и проверять SLA и точность.

Вот фрагмент кода, который считывает данные построчно из потока стандартного ввода:

In [ ]:
data = pd.DataFrame(columns=['gender','age','uid','user_json'])
s=0
for line in sys.stdin:
    data.loc[s] = line.strip().split("\t")
    s+=1

Задача вашего скрипта сделать предсказание по всем полученным строкам и выдать результат в формате json. В файле должны быть только те пользователи, у которых пол и возрастная категория изначально неизвестны, и они должны быть **отсортированы по UID по возрастанию значений лексикографически.** Пример вывода указан ниже.

In [ ]:
output = output[['uid', 'gender', 'age']]
output.sort_values(by='uid',axis = 0, ascending = True, inplace = True)
sys.stdout.write(output.to_json(orient='records'))

**Примечание**. Так как в данном проекте вам потребуется сохранять обученные модели в файле, то в вашем коде их нужно загружать явным образом. Иначе чекер не сможет запусть ваш скрипт корректно. Пример:

In [ ]:
#считать модель из файла в переменную vectorizer
import pickle
vectorizer= pickle.load(open('/data/home/test.user.bd8/TfidfVectorizer.pickle', 'rb'))

Для самопроверки вы можете локально оттестировать свой скрипт, используя следующую команду:

In [ ]:
!tail -n1000 gender_age_dataset.txt | python3 project01_gender-age.py > output.json

### Подсказки

1. Есть много различных способов решить данную задачу: можно просто хорошо поработать с урлами и доменами, можно пропарсить содержимое этих урлов (заголовки, текст и т.д.) и воспользоваться неким векторизатором типа TF\*IDF для генерации дополнительных фич, которые уже в дальнейшем вы подадите на вход ML-алгоритму, можно сделать тематическое моделирование (LDA, BigARTM) сайтов и использовать одну или несколько тем в качестве фич.

2. Возможно, что данные грязные и их нужно дополнительно обработать. Спецсимволы, кириллические домены? Уделите этому этапу достаточно времени: здесь чистота датасета важнее, чем выбор алгоритма.

3. Часто бывает, что лучшее решение с точки зрения результата — оно же самое простое. Попробуйте сначала простые способы, простые алгоритмы, прежде чем переходить к тяжёлой артиллерии. Один из вариантов — начать с небольшого RandomForest.

4. Вам почти наверняка понадобится что-то из пакета sklearn. [Документация](http://scikit-learn.org/stable/user_guide.html) — ваш лучший друг.

5. Вы можете сначала предсказать пол, а затем возраст, либо сразу и то, и другое. Экспериментируйте.

6. Объединяйтесь в команды. Так гораздо веселее и интереснее.

### Проверка
Проверка осуществляется из [Личного кабинета](http://lk.newprolab.com/lab/project1). До дедлайна вы будете проверять работу своего скрипта на валидационной выборке (2 000 записей). При наступлении дедлайна мы автоматически пересчитаем модели по скрытой тестовой выборке (3 000 записей). Это и будет финальным результатом.

* В поле `part of users with predicted gender + age` - указана доля пользователей, которая была предсказана от общего числа неизвестных пользователей (пример: по 3 000 был сделан прогноз, а всего было неизвестно 5 000, чекер выдаст 0.6).

* В поле `correctly predicted users / total number of users` - указана доля пользователей, которая была правильно предсказана (совпадает и пол, и возраст) от общего числа всех пользователей (пример: по 3 000 был сделан прогноз, правильно было спрогнозировано 1 500, а всего было неизвестно 5 000, чекер выдаст 0.3)

* В поле `correctly predicted users / number of predicted users` - указана доля пользователей, которая была правильно предсказана (совпадает и пол, и возраст) от общего числа предсказанных пользователей (пример: по 3 000 был сделан прогноз, из них правильно предсказано 1 500, чекер выдаст 0.5).

**Важное замечание!** Вы должны дать прогноз хотя бы по 50% пользователей, у которых изначально не указан пол и возрастная категория. Иными словами, вы можете оставить неопределенными не более 50% изначально неопределенных пользователей.

**Если доля в последнем поле превысит порог 0.28, то проект будет засчитан, при условии что выполнен SLA в 0.04 секунды**

Лучшей команде, набравшей максимальный результат, мы подарим специальный приз, о котором скажем позднее.